# Tensor Shapes in Keras

Let's take a look at defining models in Keras, using the functional API, and how to test the model using Numpy arrays. First, let's take a quick look at Numpy's `ndarray` type and some pecularities related to it.


## Numpy ndarray

When using Numpy objects with TensorFlow, it is sometimes slightly confusing that depending on how a shape of an [ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) object is defined, it can behave differently. For example, a vector defined as `np.array([1, 2, 3])`, with a shape (3, ) can behave as a row- or column vector. Following simple Python program highlights this:

```
import numpy as np
arr = np.array([1, 2, 3])
mat = np.array([[1, 0, 0], [0, 2, 0], [0, 0, 3]])
result1 = arr@mat
result2 = mat@arr
```

, where `@` indicates matrix multiplication. When we calculate `arr @ mat`, `arr` behaves as follows:

$$
\begin{bmatrix}
1 & 2 & 3
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0 \\
0 & 2 & 0 \\
0 & 0 & 3 \\
\end{bmatrix} = 
\begin{bmatrix}
1 & 4 & 9
\end{bmatrix}
$$

However, when we calculate `mat @ arr`, `arr` behaves as follows:

$$
\begin{bmatrix}
1 & 0 & 0 \\
0 & 2 & 0 \\
0 & 0 & 3 \\
\end{bmatrix}
\begin{bmatrix}
1 \\ 2 \\ 3
\end{bmatrix} = 
\begin{bmatrix}
1 \\ 4 \\ 9
\end{bmatrix}
$$

In both of the above cases, shapes of both `result1` and `result2` would be (3, ). Now, if we define the shape of the `arr` to be (3,1), then only `mat @ arr` would work, and the resulting shape would be (3, 1).

Therefore, an array which shape is (n, ) can be thought of being a 1D object that has n-number of elements, and it can behave as either a row- or column vector in matrix calculations. On the other hand, a vector with a shape (3, 1) or (1, 3) is a 2D object that has clearly defined shape.

## Keras Input Layer

When building a model, using the functional API, we typically use `tf.keras.Input`. Now, let's see what the documentation has to say regarding this element.

> `Input()` is used to instantiate a Keras tensor.
A Keras tensor is a symbolic tensor-like object, which we augment with certain attributes that allow us to build a Keras model just by knowing the inputs and outputs of the model.

>For instance, if a, b and c are Keras tensors, it becomes possible to do: `model = Model(input=[a, b], output=c)`

One of the parameters, when building an `Input` layer, is the `shape`of the input:

> **shape** : A shape tuple (integers), not including the batch size. For instance, shape=(32,) indicates that the expected input will be batches of 32-dimensional vectors. Elements of this tuple can be None; 'None' elements represent dimensions where the shape is not known. 

[Source](https://www.tensorflow.org/api_docs/python/tf/keras/Input)

For more information regarding tensor ranks, take a look at [source](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/resources/dims_types.md)

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Reduce TF verbosity
import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel('INFO') # Reduce TF verbosity
print(f"tensorflow version: {tf.__version__}")

import numpy as np

tensorflow version: 2.11.0


# Keras Models with Numpy Input Vectors

Here we test what input shapes different kinds of simple Keras models expect to get. Hopefully
this will shed some light on how to test Keras models using numpy data.

In [2]:
# Numpy test data
scalar = 3.0
arr = np.array([1, 2, 3])
row_vector = np.array([1, 2, 3]).reshape(1, 3)
column_vector = np.array([1, 2, 3]).reshape(3, 1)

print(f'Scalar: {scalar}')
print(f'arr, shape: {arr.shape}, data: {arr}')
print(f'row_vector, shape: {row_vector.shape}, data: {row_vector}')
print(f'column_vector, shape: {column_vector.shape}, data: {column_vector}')

Scalar: 3.0
arr, shape: (3,), data: [1 2 3]
row_vector, shape: (1, 3), data: [[1 2 3]]
column_vector, shape: (3, 1), data: [[1]
 [2]
 [3]]


In [3]:
# Function to output the results
def model_output(model, input_type, input):
    print(f'EXECUTING MODEL WITH INPUT TYPE: {input_type} -> shape: {input.shape}')
    
    result = np.array([0])
    try:
        result = model(input).numpy()
    except Exception as exc:
        print(f'Failed: {exc}')
        print('-------------------------------------------------------------------')
        return
    
    print(f'result shape: {result.shape}, result: {result}')
    print('-------------------------------------------------------------------')
    

In [4]:
# Keras model that multiplies all the values with 2.0. Since no linear algebra calculations are done,
# any array or vector shape should work
inputs = tf.keras.Input(shape=(3,))
outputs = 2*inputs
model1 = tf.keras.Model(inputs=inputs, outputs=outputs, name='model 1')

print('--- MODEL INFORMATION ---')
print(model1.summary())

--- MODEL INFORMATION ---
Model: "model 1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 tf.math.multiply (TFOpLambd  (None, 3)                0         
 a)                                                              
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
# All input shapes work
model_output(model1, "arr", arr)
model_output(model1, "row_vector", row_vector)
model_output(model1, "column_vector", column_vector)

EXECUTING MODEL WITH INPUT TYPE: arr -> shape: (3,)
result shape: (3,), result: [2. 4. 6.]
-------------------------------------------------------------------
EXECUTING MODEL WITH INPUT TYPE: row_vector -> shape: (1, 3)
result shape: (1, 3), result: [[2. 4. 6.]]
-------------------------------------------------------------------
EXECUTING MODEL WITH INPUT TYPE: column_vector -> shape: (3, 1)
result shape: (3, 1), result: [[2.]
 [4.]
 [6.]]
-------------------------------------------------------------------


In [6]:
# Keras model that calculates dot product between the input tensor and transpose of the input tensor.
# output = dot(inputs, inputs'). This model should work with arrays and row- and column- vectors, but
# the outcome is different depending on the shape
inputs = tf.keras.Input(shape=(3,))
outputs = tf.tensordot(inputs, tf.transpose(inputs), axes=1)
model2 = tf.keras.Model(inputs=inputs, outputs=outputs, name='model 2')

print('--- MODEL INFORMATION ---')
print(model2.summary())

--- MODEL INFORMATION ---
Model: "model 2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 tf.compat.v1.transpose (TFOpLa  (3, None)           0           ['input_2[0][0]']                
 mbda)                                                                                            
                                                                                                  
 tf.tensordot (TFOpLambda)      (None, None)         0           ['input_2[0][0]',                
                                                                  'tf.compat.v1.transpose[0][0]'] 
                                                                  

In [7]:
# All vector input shapes work
model_output(model2, "arr", arr)
model_output(model2, "row_vector", row_vector)
model_output(model2, "column_vector", column_vector)

EXECUTING MODEL WITH INPUT TYPE: arr -> shape: (3,)
result shape: (), result: 14.0
-------------------------------------------------------------------
EXECUTING MODEL WITH INPUT TYPE: row_vector -> shape: (1, 3)
result shape: (1, 1), result: [[14.]]
-------------------------------------------------------------------
EXECUTING MODEL WITH INPUT TYPE: column_vector -> shape: (3, 1)
result shape: (3, 3), result: [[1. 2. 3.]
 [2. 4. 6.]
 [3. 6. 9.]]
-------------------------------------------------------------------


In [8]:
inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4)(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model3 = tf.keras.Model(inputs=inputs, outputs=outputs, name='model 3')

print('--- MODEL INFORMATION ---')
print(model3.summary())

--- MODEL INFORMATION ---
Model: "model 3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 4)                 16        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# Only the row-vector, with shape (1, 3), works with the network with a dense layer connected
# to the input layer.
model_output(model3, "arr", arr)
model_output(model3, "row_vector", row_vector)
model_output(model3, "column_vector", column_vector)

EXECUTING MODEL WITH INPUT TYPE: arr -> shape: (3,)
Failed: Exception encountered when calling layer 'model 3' (type Functional).

Input 0 of layer "dense" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (3,)

Call arguments received by layer 'model 3' (type Functional):
  • inputs=tf.Tensor(shape=(3,), dtype=int64)
  • training=None
  • mask=None
-------------------------------------------------------------------
EXECUTING MODEL WITH INPUT TYPE: row_vector -> shape: (1, 3)
result shape: (1, 1), result: [[4.2222095]]
-------------------------------------------------------------------
EXECUTING MODEL WITH INPUT TYPE: column_vector -> shape: (3, 1)
Failed: Exception encountered when calling layer 'model 3' (type Functional).

Input 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (3, 1)

Call arguments received by layer 'model 3' (type Functional):
  • inputs=tf.Tensor